<a href="https://colab.research.google.com/github/Diego-LeivaC/Quiz_Project_Leiva/blob/main/market-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following notebook describes....

**API from Kaggle and Data Loading**

In [1]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:
import os
os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxx"
!kaggle datasets download -d harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows


Dataset URL: https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows
License(s): CC0-1.0
imdb-dataset-of-top-1000-movies-and-tv-shows.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip *.zip

Archive:  imdb-dataset-of-top-1000-movies-and-tv-shows.zip
replace imdb_top_1000.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: imdb_top_1000.csv       


In [4]:
import pandas as pd

data = pd.read_csv("imdb_top_1000.csv")
data.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


**Data analysis**

In [5]:
data.shape

(1000, 16)

There are 1000 rows and 16 columns, meaning that there are exactly 1000 films

In [6]:
data.columns

Index(['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')

The column 'Series_Title' refers to the films name, let's see if there are null values

In [7]:
data['Series_Title'].isnull().sum()

np.int64(0)

All movies are completed

The columns: 'Star1', 'Star2', 'Star3', 'Star4' are the actor names, let's see again if there are null values.

In [8]:
data[['Star1','Star2','Star3','Star4']].isnull().sum()

,0
Star1,0
Star2,0
Star3,0
Star4,0


All actors are completed

Furthermore, considering that each movie is to be considered a basket, and actors listed under the Star1, Star2, Star3 and Star4 fields as items. I transform each movie in a list of actors, using the following code.

In [9]:
transactions = data[['Star1','Star2','Star3','Star4']].values.tolist()

In [10]:
import random
print(random.sample(transactions, 5))

[['Winona Ryder', 'Gena Rowlands', 'Lisanne Falk', 'Alan Randolph Scott'], ['Robert De Niro', 'Kevin Bacon', 'Brad Pitt', 'Jason Patric'], ['Charlton Heston', 'Orson Welles', 'Janet Leigh', 'Joseph Calleia'], ['Farhan Akhtar', 'Sonam Kapoor', 'Pawan Malhotra', 'Art Malik'], ['Robert De Niro', 'Chazz Palminteri', 'Lillo Brancato', 'Francis Capra']]


The attribute transactions is in a market-basket format

**A-priori algorithm**

We begin with the A-Priori Algorithm, which works by eliminating most
large sets as candidates by looking first at smaller sets and recognizing that a
large set cannot be frequent unless all its subsets are.

In [11]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

transaction_encoder = TransactionEncoder() # Creates an object that can transform data (actor names) to a mathematic format.
te_array = transaction_encoder.fit(transactions).transform(transactions) # Convert the variable "transactions", transforming it in Falso/True.
data_encoded = pd.DataFrame(te_array, columns=transaction_encoder.columns_) # Convert the matrix into a table



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [12]:
data_encoded.head()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,Aamir Bashir,Aamir Khan,Aaron Eckhart,Aaron Taylor-Johnson,Abdel Ahmed Ghili,Abhay Deol,Abigail Breslin,Abraham Attah,Adam Baldwin,Adam Driver,...,Ziyi Zhang,Zoe Saldana,Zooey Deschanel,Zoë Kravitz,Álvaro Guerrero,Çetin Tekindor,Émile Vallée,Éric Toledano,Ömer Faruk Sorak,Özge Özberk
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [13]:
frequent_combinations = apriori(data_encoded, min_support=0.01, use_colnames=True) # Execution of the a-priori algorithm,
# only returns combinations that appears in at least 0.01 (1%) of films, showing the name of the actors.
frequent_combinations.sort_values("support", ascending=False).head() # Orders by frequency, showing the first results (most frequent actors)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,support,itemsets
7,0.017,(Robert De Niro)
8,0.014,(Tom Hanks)
0,0.013,(Al Pacino)
1,0.012,(Brad Pitt)
3,0.012,(Clint Eastwood)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Robert De Niro appears in 1.7% of films, meaning that De Niro appears in 17 films of the entire dataset (1000). The same logic applies to the rest of actors.

Filter to see only couples or threesomes

In [14]:
# Creation of a column with the length of each combination
frequent_combinations['length'] = frequent_combinations['itemsets'].apply(lambda x: len(x))

# Filter to show only combinations of 2 or more actors and displayed the 15 best.
frequent_combinations[frequent_combinations['length'] >= 2].sort_values("support", ascending=False).head(15)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,support,itemsets,length


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

No results

To find couples or more, I need to be more flexible. I try lowering the support to 0.002 (which means it's enough that they've appeared together in 2 movies):

In [18]:
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Lowering the support to 0.002 (2 out of every 1000 films)
frequent_combinations = apriori(data_encoded, min_support=0.002, use_colnames=True)

# Creation of the column length
frequent_combinations['length'] = frequent_combinations['itemsets'].apply(lambda x: len(x))

# Filtering and sorting
couples = frequent_combinations[frequent_combinations['length'] >= 2].sort_values("support", ascending=False)
couples.head(15)

,support,itemsets,length
689,0.006,"(Daniel Radcliffe, Rupert Grint)",2
687,0.005,"(Daniel Radcliffe, Emma Watson)",2
774,0.005,"(Daniel Radcliffe, Emma Watson, Rupert Grint)",3
701,0.005,"(Emma Watson, Rupert Grint)",2
723,0.004,"(Joe Pesci, Robert De Niro)",2
760,0.004,"(Tim Allen, Tom Hanks)",2
670,0.003,"(Chris Evans, Joe Russo)",2
663,0.003,"(Carrie Fisher, Mark Hamill)",2
698,0.003,"(Orlando Bloom, Elijah Wood)",2
695,0.003,"(Diane Keaton, Woody Allen)",2


In [19]:
# maximum support for any pair found
print("Maximum support in pairs:", frequent_combinations[frequent_combinations['length'] >= 2]['support'].max())

Maximum support in pairs: 0.006
